# RandomForests

## Setup Coiled

In [2]:
import coiled
cluster = coiled.Cluster(10, configuration="coiled/dask-examples")

from dask.distributed import Client
client = Client(cluster)
client.dashboard_link

'http://ec2-3-22-166-88.us-east-2.compute.amazonaws.com:8787/status'

## Set up problem


In [8]:
from joblib import parallel_backend
import numpy as np
from sklearn.datasets import load_digits
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

digits = load_digits()
clf = RandomForestClassifier(n_estimators=45000,verbose=1)


## We could run locally

In principle we want to run the following:

```python
with parallel_backend('dask'):
    clf.fit(digits.data, digits.target)
```

But then our local session will be in rapid communication with all of the workers.  This is fine if we're running this from the cloud, but if we're running this from our laptop then it's better to run this on a worker.

In [9]:
def train(clf, X, y):
    with parallel_backend('dask'):
        clf.fit(digits.data, digits.target)
    
    return clf

In [10]:
%%time

future = client.submit(train, clf, digits.data, digits.target)
s = client.submit(str, future)
s.result()

CPU times: user 267 ms, sys: 44.9 ms, total: 312 ms
Wall time: 1min 4s


'RandomForestClassifier(n_estimators=45000, verbose=1)'

distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client
_GatheringFuture exception was never retrieved
future: <_GatheringFuture finished exception=CancelledError()>
asyncio.exceptions.CancelledError
